# Spark Streaming이 돌아갈 수 있는 환경 설정을 반드시 해주세요.

## 1. 실시간 데이터 처리 개념과 소켓 열기
* 실시간 데이터 스트리밍을 위한 네트워크 소켓을 설정하여 데이터를 Spark에 전달할 수 있도록 합니다. 이 예제를 통해 소켓을 설정할 수 있습니다.

### Netcat을 이용한 소켓 열기
* 다음 명령어를 사용하여 로컬 호스트의 9999 포트에서 데이터를 지속적으로 수신합니다.

# Netcat 설치
* sudo apt update
* sudo apt install netcat

# 소켓 열기
* nc -lk 9999


### TODO: 주석과 참고 페이지를 참고하여 실시간 스트리밍 데이터 처리를 가능하도록 완성해주세요.

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import col

# SparkSession 생성
# Spark 애플리케이션을 위한 진입점으로 SparkSession을 생성합니다.
# - appName("FileSourceAggregation"): 애플리케이션 이름을 설정합니다.
# - config("spark.streaming.stopGracefullyOnShutdown", "true"): 셧다운 시 스트리밍을 안전하게 종료합니다.
# 참고: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.SparkSession.html
spark = _____________ \
    .builder \
    .appName("FileSourceAggregation") \
    .config("spark.streaming.stopGracefullyOnShutdown", "true") \
    .getOrCreate()




In [ ]:
# 데이터 스키마 정의
# JSON 파일에서 읽어올 데이터의 스키마를 설정합니다.
# - amount: IntegerType, 제품 금액을 나타내는 정수형 필드
# - product_id: StringType, 제품 ID를 나타내는 문자열 필드
# 참고: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.types.StructType.html
schema = StructType([
    StructField("amount", IntegerType()),
    StructField("product_id", StringType())
])

In [ ]:
# 파일 소스에서 데이터 읽기
# 지정된 경로의 JSON 파일을 스트리밍 소스로 설정하여 데이터를 읽어옵니다.
# - format("json"): JSON 형식의 파일을 읽어옵니다.
# - schema(schema): 미리 정의한 스키마를 적용합니다.
# 참고: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrameReader.format.html
# 참고: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.SparkSession.readStream.html
file_path = "_____________"  # JSON 파일이 위치할 디렉터리 경로 설정
events = spark \
    ._______________ \
    .format("json") \
    .schema(schema) \
    .load(file_path)

In [ ]:
# 데이터 집계: product_id 별로 amount의 합계를 구하여 sum(amount)라는 열로 반환합니다.
# 각 product_id에 대해 amount의 합계를 계산하여 total_amount라는 열로 반환합니다.
# 참고: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.groupBy.html
window_agg_df = events.____________("product_id").sum("amount").____________("sum(amount)", "total_amount")


In [ ]:
import time

# 스트리밍 쿼리를 시작
# - outputMode("update"): 업데이트된 데이터만 출력합니다.
# - format("console"): 콘솔에 출력하도록 설정합니다.
# - option("truncate", "false"): 출력 결과가 잘리지 않도록 설정합니다.
# 참고: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.streaming.DataStreamWriter.outputMode.html
# 참고: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.writeStream.html

query = window_agg_df \
    .____________ \
    .____________("update") \
    .format("console") \
    .option("truncate", "false") \
    .start()
    
# 10초 대기
# 데이터 처리가 이루어질 시간을 주기 위해 10초 동안 대기합니다.
time.sleep(10)

# 10초 후 스트리밍 쿼리 중지
# query.stop()으로 스트리밍 쿼리를 안전하게 종료합니다.
query.stop()
